# Создание sql-запросов к базе данных сервиса StackOverflow

В проекте мы напишем sql-запросы для решения задач самостоятельного проекта в допкурсе Продвинутый SQL. Результаты запросов проверены тренажером Яндекс.Практикума

Проект выполним, используя базу данных StackOverflow — сервиса вопросов и ответов о программировании. 

StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
Мы будем работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах можно найти информацию и о более поздних оценках, которые эти посты получили.

Данные в базе находятся в таблицах

 - badges
 - post_types
 - posts
 - users
 - vote_types
 - votes


**Задачи данной работы:**

 - выгрузить информацию из базы данных в файлы `csv`
 
 - исследовать таблицы
 
 - написать SQL-запросы
 

**Таким образом работа пройдет по следующим этапам:**

 - создадим коннекцию к базе
 - напишем код для выгрузки данных 
 - напишем SQL-запросы
  

## Запрос к базе данных

 - Создадим запрос к базе данных и сохраним информацию в csv-файл

In [1]:
#!pip install ipython-sql               # импортируем библиотеки
#!pip3 install psycopg2

import pandas as pd
from sqlalchemy import create_engine  

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-advanced-sql'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
                                               

In [3]:
query = '''                  
           SELECT *
           FROM stackoverflow.badges
        '''
badges_raw = pd.io.sql.read_sql(query, con = engine)
badges_raw.to_csv('badges.csv', index = False)
badges = badges_raw
badges.head()

,id,name,user_id,creation_date
0,82947,Teacher,994,2008-09-15 08:55:04
1,82949,Teacher,3893,2008-09-15 08:55:04
2,82950,Teacher,4591,2008-09-15 08:55:04
3,82951,Teacher,5196,2008-09-15 08:55:04
4,82952,Teacher,2635,2008-09-15 08:55:04


In [4]:
badges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330130 entries, 0 to 330129
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             330130 non-null  int64         
 1   name           330128 non-null  object        
 2   user_id        330130 non-null  int64         
 3   creation_date  330130 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 10.1+ MB


Поля таблицы
 - идентификатор значка, первичный ключ таблицы
 - название значка
 - идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users
 - дата присвоения значка

In [5]:
query = '''                  
           SELECT *
           FROM stackoverflow.post_types
        '''
post_types_raw = pd.io.sql.read_sql(query, con = engine)
post_types_raw.to_csv('post_types.csv', index = False)
post_types = post_types_raw
post_types.head()

,id,type
0,1,Question
1,2,Answer


In [6]:
post_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2 non-null      int64 
 1   type    2 non-null      object
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


Поля таблицы
- идентификатор поста, первичный ключ таблицы
- тип поста

In [7]:
query = '''                  
           SELECT *
           FROM stackoverflow.posts
        '''
posts_raw = pd.io.sql.read_sql(query, con = engine)
posts_raw.to_csv('posts', index = False)
posts = posts_raw
posts.head()

,id,title,creation_date,favorites_count,last_activity_date,last_edit_date,user_id,parent_id,post_type_id,score,views_count
0,4,Convert Decimal to Double?,2008-07-31 21:42:53,41,2018-07-02 17:55:27.247,2018-07-02 17:55:27,8,0,1,573,37080
1,6,Percentage width child element in absolutely p...,2008-07-31 22:08:09,10,2016-03-19 06:10:52.170,2016-03-19 06:05:48,9,0,1,256,16306
2,7,None,2008-07-31 22:17:58,0,2017-12-16 05:06:57.613,2017-12-16 05:06:58,9,4,2,401,0
3,9,How do I calculate someone's age in C#?,2008-07-31 23:41:00,399,2018-07-25 11:57:14.110,2018-04-21 17:48:14,1,0,1,1743,480476
4,11,Calculate relative time in C#,2008-07-31 23:55:38,529,2018-07-05 04:00:56.633,2017-06-04 15:51:20,1,0,1,1348,136033


In [8]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243796 entries, 0 to 243795
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  243796 non-null  int64         
 1   title               52345 non-null   object        
 2   creation_date       243796 non-null  datetime64[ns]
 3   favorites_count     243796 non-null  int64         
 4   last_activity_date  243796 non-null  object        
 5   last_edit_date      87139 non-null   datetime64[ns]
 6   user_id             243796 non-null  int64         
 7   parent_id           243796 non-null  int64         
 8   post_type_id        243796 non-null  int64         
 9   score               243796 non-null  int64         
 10  views_count         243796 non-null  int64         
dtypes: datetime64[ns](2), int64(7), object(2)
memory usage: 20.5+ MB


Поля таблицы
 - идентификатор поста, первичный ключ таблицы
 - заголовок поста
 - дата создания поста
 - число, которое показывает, сколько раз пост добавили в «Закладки»
 - дата последнего действия в посте, например комментария
 - дата последнего изменения поста
 - идентификатор пользователя, который создал пост, внешний ключ к таблице users
 - если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
 - идентификатор типа поста, внешний ключ к таблице post_types
 - количество очков, которое набрал пост
 - количество просмотров

In [9]:
query = '''                  
           SELECT *
           FROM stackoverflow.users
        '''
users_raw = pd.io.sql.read_sql(query, con = engine)
users_raw.to_csv('users', index = False)
users = users_raw
users.head()

,id,creation_date,display_name,last_access_date,location,reputation,views
0,1,2008-07-31 14:22:31,Jeff Atwood,2018-08-29 02:34:23,"El Cerrito, CA",44300,408587
1,2,2008-07-31 14:22:31,Geoff Dalgas,2018-08-23 17:31:56,"Corvallis, OR",3491,23966
2,3,2008-07-31 14:22:31,Jarrod Dixon,2018-08-30 20:56:24,"Raleigh, NC, United States",13418,24396
3,4,2008-07-31 14:22:31,Joel Spolsky,2018-08-14 22:18:15,"New York, NY",28768,73755
4,5,2008-07-31 14:22:31,Jon Galloway,2018-08-29 16:48:36,"San Diego, CA",39172,11700


In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                24044 non-null  int64         
 1   creation_date     24044 non-null  datetime64[ns]
 2   display_name      24042 non-null  object        
 3   last_access_date  24044 non-null  datetime64[ns]
 4   location          15599 non-null  object        
 5   reputation        24044 non-null  int64         
 6   views             24044 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 1.3+ MB


Поля таблицы
 - идентификатор пользователя, первичный ключ таблицы
 - дата регистрации пользователя
 - имя пользователя
 - дата последнего входа
 - местоположение
 - очки репутации, которые получают за хорошие вопросы и полезные ответы
 - число просмотров профиля пользователя

In [11]:
query = '''                  
           SELECT *
           FROM stackoverflow.vote_types
        '''
vote_types_raw = pd.io.sql.read_sql(query, con = engine)
vote_types_raw.to_csv('vote_types', index = False)
vote_types = vote_types_raw
vote_types.head()

,id,name
0,1,AcceptedByOriginator
1,2,UpMod
2,3,DownMod
3,4,Offensive
4,5,Favorite


In [12]:
vote_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      15 non-null     int64 
 1   name    15 non-null     object
dtypes: int64(1), object(1)
memory usage: 368.0+ bytes


 - идентификатор типа голоса, первичный ключ
 - название метки

In [13]:
query = '''                  
           SELECT *
           FROM stackoverflow.votes
        '''
votes_raw = pd.io.sql.read_sql(query, con = engine)
votes_raw.to_csv('votes', index = False)
votes = votes_raw
votes.head()

,id,post_id,user_id,bounty_amount,vote_type_id,creation_date
0,434364,146794,4918,NaN,5,2008-10-03
1,434366,135535,16511,NaN,3,2008-10-03
2,434372,162181,12818,NaN,1,2008-10-03
3,434380,21965,4002,NaN,1,2008-10-03
4,434401,165975,8161,NaN,2,2008-10-03


In [14]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58790 entries, 0 to 58789
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             58790 non-null  int64  
 1   post_id        58790 non-null  int64  
 2   user_id        58790 non-null  int64  
 3   bounty_amount  220 non-null    float64
 4   vote_type_id   58790 non-null  int64  
 5   creation_date  58790 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 2.7+ MB


Поля таблицы
 - идентификатор голоса, первичный ключ
 - идентификатор поста, внешний ключ к таблице posts
 - идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users
 - сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту
 - идентификатор типа голоса, внешний ключ к таблице vote_types
 - дата назначения голоса


In [15]:
tables = [badges, post_types, posts, users, vote_types, votes]
for i in tables:
    print(i.duplicated().sum())

0
0
0
0
0
0


**Вывод**

 - типы данных представлены в корректном виде
 - дубликатов нет
 - есть пропуски в таблицах 
    - posts в столбцах title, last_edit_day Возможно у поста нет названия и на момент выгрузки данных отсутствует дата последнего изменения. Оставим пропуски без изменения
    - users в столбце location Пропущена информация о местоположении пользователя. Наверно, данные пользователи хотели быть законсперированы.  Оставим пропуски без изменения
    - votes в столбце bounty_amount. Вознаграждение, которое назначают для привлечения внимания к посту. Очень редко пользователи сервиса используют этот метод. Практически все значения пропущены. Данные пропуски также оставим без изменения

## SQL-запросы

**Первая часть**

Задача 1

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [16]:
query_1 = ''' SELECT COUNT(p.id)
              FROM stackoverflow.posts p
              WHERE p.post_type_id = 1 AND p.score > 300 OR p.favorites_count >= 100
'''
result_1 = pd.read_sql(query_1, con = engine)
print(result_1)

   count
0   1355


Задача 2 

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [17]:
query_2 = ''' SELECT ROUND(AVG(c.cnt))::integer
              FROM
              (SELECT COUNT(p.id) AS cnt,
              EXTRACT(DAY FROM p.dt) AS days
             FROM      
            (SELECT id,
             CAST(creation_date AS date) AS dt
            FROM stackoverflow.posts 
            WHERE post_type_id = 1 AND (creation_date::date BETWEEN '2008-11-01' AND '2008-11-18')) AS p
            GROUP BY days) AS c
'''
result_2 = pd.read_sql(query_2, con = engine)
print(result_2)

   round
0    383


Задача 3


Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [18]:
query_3 = ''' SELECT COUNT(DISTINCT u.id)
              FROM stackoverflow.users u
              JOIN stackoverflow.badges b ON u.id = b.user_id
              WHERE u.creation_date::date = b.creation_date::date
'''
result_3 = pd.read_sql(query_3, con = engine)
print(result_3)

   count
0   7047


Задача 4


Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [19]:
query_4 = '''SELECT COUNT(DISTINCT p.id)
            FROM stackoverflow.votes v
            JOIN stackoverflow.posts p ON v.post_id = p.id
            JOIN stackoverflow.users u ON p.user_id = u.id
            WHERE u.display_name = 'Joel Coehoorn'
'''
result_4 = pd.read_sql(query_4, con = engine)
print(result_4)

   count
0     12


Задача 5

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [20]:
query_5 = '''SELECT *,
             ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
             FROM stackoverflow.vote_types
             ORDER BY id
'''
result_5 = pd.read_sql(query_5, con = engine)
print(result_5)

    id                   name  rank
0    1   AcceptedByOriginator    15
1    2                  UpMod    14
2    3                DownMod    13
3    4              Offensive    12
4    5               Favorite    11
5    6                  Close    10
6    7                 Reopen     9
7    8            BountyStart     8
8    9            BountyClose     7
9   10               Deletion     6
10  11             Undeletion     5
11  12                   Spam     4
12  13        InformModerator     3
13  14        ModeratorReview     2
14  15  ApproveEditSuggestion     1


Задача 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [21]:
query_6 = '''SELECT u.id AS us,
             COUNT(vt.id) AS cl
             FROM stackoverflow.users u
             JOIN stackoverflow.votes v ON u.id = v.user_id
             JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
             WHERE vt.name = 'Close'
             GROUP BY 1
             ORDER BY cl DESC, us DESC
             LIMIT 10
'''
result_6 = pd.read_sql(query_6, con = engine)
print(result_6)

      us  cl
0  20646  36
1  14728  36
2  27163  29
3  41158  24
4  24820  23
5   9345  23
6   3241  23
7  44330  20
8  38426  19
9  19074  19


Задача 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
 - идентификатор пользователя;
 - число значков;
 - место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [22]:
query_7 = '''WITH r AS (SELECT user_id,
            COUNT(name)  AS bad
             FROM stackoverflow.badges
             WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
             GROUP BY 1)
            SELECT  r.user_id,
           r.bad,
            DENSE_RANK()OVER(ORDER BY r.bad DESC)
            FROM r
           ORDER BY 2 DESC, 1
            LiMIT 10
'''
result_7 = pd.read_sql(query_7, con = engine)
print(result_7)

   user_id  bad  dense_rank
0    22656  149           1
1    34509   45           2
2     1288   40           3
3     5190   31           4
4    13913   30           5
5      893   28           6
6    10661   28           6
7    33213   25           7
8    12950   23           8
9    25222   20           9


Задача 8


Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
 - заголовок поста;
 - идентификатор пользователя;
 - число очков поста;
 - среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [23]:
query_8 = '''SELECT title,
             user_id,
             score,
             ROUND(AVG(score) OVER(PARTITION BY user_id)) AS avg_sc
             FROM stackoverflow.posts
             WHERE score != 0 AND title IS NOT NULL
             LIMIT 5
'''
result_8 = pd.read_sql(query_8, con = engine)
print(result_8)

                                               title  user_id  score  avg_sc
0            Diagnosing Deadlocks in SQL Server 2005        1     82   573.0
1            How do I calculate someone's age in C#?        1   1743   573.0
2  Why doesn't IE7 copy <pre><code> blocks to the...        1     37   573.0
3                      Calculate relative time in C#        1   1348   573.0
4  Wrapping StopWatch timing with a delegate or l...        1     92   573.0


Задача 9

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [24]:
query_9 = '''SELECT title
             FROM stackoverflow.posts
             WHERE user_id IN
             (SELECT cnt.user_id
             FROM
             (SELECT  user_id,
                      COUNT(id)  AS badg
              FROM stackoverflow.badges
              GROUP BY 1) AS cnt
             WHERE cnt.badg > 1000) AND title IS NOT NULL
'''
result_9 = pd.read_sql(query_9, con = engine)
print(result_9)

                                               title
0  What's the strangest corner case you've seen i...
1  What's the hardest or most misunderstood aspec...
2       What are the correct version numbers for C#?
3               Project management to go with GitHub


Задача 10

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в  зависимости от количества просмотров их профилей:
 - пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
 - пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
 - пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [25]:
query_10 = '''
SELECT id,
        views,
        CASE
            WHEN views >= 350 THEN 1
            WHEN views < 350 AND views >= 100 THEN 2
            ELSE 3
         END   
FROM stackoverflow.users
WHERE views != 0 AND location LIKE 'United States'
GROUP BY 1, 2
'''
result_10 = pd.read_sql(query_10, con = engine)
print(result_10)

         id  views  case
0     26540     65     3
1     20710     66     3
2     28987     97     3
3     32893     14     3
4     15559     51     3
...     ...    ...   ...
1475   3746     75     3
1476    958    165     2
1477  18356    341     2
1478  10591    296     2
1479  37810     20     3

[1480 rows x 3 columns]


Задача 11

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [26]:
query_11 = '''WITH r AS (SELECT prev.id,
              prev.case,
              prev.views,    
              DENSE_RANK() OVER(PARTITION BY prev.case ORDER BY prev.views DESC) AS rang
              FROM     
              (SELECT id,
                     views,
                   CASE
                     WHEN views >= 350 THEN 1
                     WHEN views < 350 AND views >= 100 THEN 2
                     ELSE 3
                   END   
            FROM stackoverflow.users
           WHERE location LIKE 'United States' AND views != 0
           GROUP BY 1, 2) AS prev)
           SELECT r.id,
                  r.case,
                  r.views
           FROM r
           WHERE r.rang =1
           GROUP BY 1,2,3
           ORDER BY 3 DESC, 1
'''
result_11 = pd.read_sql(query_11, con = engine)
print(result_11)

      id  case  views
0   1228     1  58635
1   9585     2    349
2  15079     2    349
3  33437     2    349
4   3469     3     99
5   4829     3     99
6  22732     3     99


Задача 12


Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
 - номер дня;
 - число пользователей, зарегистрированных в этот день;
 - сумму пользователей с накоплением.

In [27]:
query_12 = '''
           WITH s AS (SELECT EXTRACT(DAY FROM CAST(creation_date AS date)) AS days,
                     COUNT(id) OVER(PARTITION BY creation_date::date) AS cnt,
                     COUNT(id) OVER(ORDER BY creation_date::date) AS cumm       
                     FROM stackoverflow.users
                     WHERE CAST(creation_date AS date) BETWEEN '2008-11-01' AND '2008-11-30')
                    SELECT s.days::integer,
                            s.cnt,
                           s.cumm
                           FROM s
                          GROUP BY 1,2,3
                          ORDER BY 1
'''
result_12 = pd.read_sql(query_12, con = engine)
print(result_12)

    days  cnt  cumm
0      1   34    34
1      2   48    82
2      3   75   157
3      4  192   349
4      5  122   471
5      6  132   603
6      7  104   707
7      8   42   749
8      9   45   794
9     10   93   887
10    11  113  1000
11    12  113  1113
12    13   96  1209
13    14   89  1298
14    15   42  1340
15    16   32  1372
16    17   84  1456
17    18   89  1545
18    19  107  1652
19    20   95  1747
20    21   81  1828
21    22   40  1868
22    23   50  1918
23    24   84  2002
24    25  104  2106
25    26   98  2204
26    27   71  2275
27    28   56  2331
28    29   44  2375
29    30   33  2408


Задача 13


Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
 - идентификатор пользователя;
 - разницу во времени между регистрацией и первым постом.

In [28]:
query_12 = '''
           WITH s AS (SELECT user_id,
       MIN(creation_date) OVER(PARTITION BY user_id) AS md
FROM stackoverflow.posts)    
SELECT s.user_id,
       (s.md - u.creation_date) AS diff
FROM s
LEFT JOIN stackoverflow.users u ON s.user_id = u.id
GROUP BY 1,2
'''
result_12 = pd.read_sql(query_12, con = engine)
print(result_12)

       user_id             diff
0            1  0 days 09:18:29
1            2  0 days 14:37:03
2            3  3 days 16:17:09
3            4 15 days 05:44:22
4            5  1 days 14:57:51
...        ...              ...
18844  3644960  0 days 00:00:00
18845  3742716 66 days 21:15:48
18846  3930756  8 days 02:11:55
18847  4020932 47 days 18:30:34
18848  5696608  0 days 00:24:59

[18849 rows x 2 columns]


**Вторая часть**

Задача 14

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [29]:
query_14 = '''
SELECT SUM(views_count) AS summ,
       CAST(DATE_TRUNC('month', creation_date)AS date) AS mon
FROM stackoverflow.posts
GROUP BY 2
ORDER BY 1 DESC
'''
result_14 = pd.read_sql(query_14, con = engine)
print(result_14)

        summ         mon
0  452928568  2008-09-01
1  365400138  2008-10-01
2  221759651  2008-11-01
3  197792841  2008-12-01
4  131367083  2008-08-01
5     669895  2008-07-01


Комментарий

- Возможно, за июль неполные данные

Задача 15


Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [30]:
query_15 = '''
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE pt.type = 'Answer'
       AND p.creation_date::date <= u.creation_date::date + INTERVAL '1 month'
GROUP BY 1
HAVING COUNT(p.id) > 100
ORDER BY 1
LIMIT 10
'''
result_15 = pd.read_sql(query_15, con = engine)
print(result_15)

       display_name  count
0  1800 INFORMATION      1
1     Adam Bellaire      1
2        Adam Davis      1
3         Adam Liss      1
4              Alan      8
5             Amy B      1
6     Ben Hoffstein      1
7             Brian     15
8         Cade Roux      1
9            CesarB      1


Комментарий

- Как правило, популярному имени соответствует несколько уникальных идентификаторов

Задача 16


Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [31]:
query_16 = '''
SELECT COUNT(id) AS cnt,
       CAST(DATE_TRUNC('month', creation_date)AS date) AS mon
FROM stackoverflow.posts
WHERE user_id IN
(SELECT u.id
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
WHERE CAST(u.creation_date AS date) BETWEEN '2008-09-01' AND '2008-09-30'
      AND CAST(p.creation_date AS date) BETWEEN '2008-12-01' AND '2008-12-31')
GROUP BY 2
ORDER BY 2 DESC
'''
result_16 = pd.read_sql(query_16, con = engine)
print(result_16)

     cnt         mon
0  17641  2008-12-01
1  18294  2008-11-01
2  27171  2008-10-01
3  24870  2008-09-01
4     32  2008-08-01


Комментарий

- Пользователи, зарегистрированные в сентябре, были активны и в августе. Возможно это ошибка в данных

Задача 17

Используя данные о постах, выведите несколько полей:
 - идентификатор пользователя, который написал пост;
 - дата создания поста;
 - количество просмотров у текущего поста;
 - сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [32]:
query_17 = '''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date) AS summ
FROM stackoverflow.posts
ORDER BY 1
'''
result_17 = pd.read_sql(query_17, con = engine)
print(result_17)

        user_id       creation_date  views_count    summ
0             1 2008-07-31 23:41:00       480476  480476
1             1 2008-07-31 23:55:38       136033  616509
2             1 2008-07-31 23:56:41            0  616509
3             1 2008-08-04 02:45:08            0  616509
4             1 2008-08-04 04:31:03            0  616509
...         ...                 ...          ...     ...
243791  5696608 2008-12-23 16:00:37            0    2804
243792  5696608 2008-12-23 17:35:09            0    2804
243793  5696608 2008-12-24 01:02:48            0    2804
243794  5696608 2008-12-30 14:34:45            0    2804
243795  5696608 2008-12-30 16:32:12            0    2804

[243796 rows x 4 columns]


Комментарий

- 

Задача 18

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [33]:
query_18 = '''
WITH r AS (SELECT COUNT(begi.data) AS days,
       begi.user_id
FROM 
(SELECT COUNT(CAST(DATE_TRUNC('day', creation_date) AS date)) AS cnt,
       user_id,
       CAST(creation_date AS date) AS data
FROM stackoverflow.posts
WHERE CAST(creation_date AS date) BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY 2,3) AS begi
GROUP BY 2)

SELECT ROUND(AVG(r.days))::integer
FROM r
'''
result_18 = pd.read_sql(query_18, con = engine)
print(result_18)

   round
0      2


In [34]:
query_18 = '''
WITH r AS (SELECT COUNT(begi.data) AS days,
       begi.user_id
FROM 
(SELECT COUNT(CAST(DATE_TRUNC('day', creation_date) AS date)) AS cnt,
       user_id,
       CAST(creation_date AS date) AS data
FROM stackoverflow.posts
WHERE CAST(creation_date AS date) BETWEEN '2008-10-01' AND '2008-10-07'
GROUP BY 2,3) AS begi
GROUP BY 2)

SELECT ROUND(AVG(r.days))::integer
FROM r
'''
result_18 = pd.read_sql(query_18, con = engine)
print(result_18)

   round
0      2


Комментарий

- В среднем 2 дня. Много это или мало можно определить только после сравнения. Сопоставили с аналогичным периодом в сентябре и в октябре. Результаты совпадают. На каникулах программисты не отдыхают!

Задача 19


На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
 - номер месяца;
 - количество постов за месяц;
 - процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [35]:
query_19 = '''
WITH s AS (SELECT EXTRACT(MONTH FROM CAST(creation_date AS date)) AS mon,
        COUNT(DISTINCT id) AS cnt
FROM stackoverflow.posts
WHERE CAST(creation_date AS date) BETWEEN '2008-09-01' AND '2008-12-31'           
GROUP BY 1)
SELECT *,
     ROUND(((s.cnt::numeric / LAG(s.cnt) OVER(ORDER BY s.mon)) -1) *100, 2) AS per
FROM s
'''
result_19 = pd.read_sql(query_19, con = engine)
print(result_19)

    mon    cnt    per
0   9.0  70371    NaN
1  10.0  63102 -10.33
2  11.0  46975 -25.56
3  12.0  44592  -5.07


Комментарий



Задача 20

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
 - номер недели;
 - дата и время последнего поста, опубликованного на этой неделе.

In [36]:
query_20 = '''
WITH r AS (SELECT creation_date,
       EXTRACT(WEEK FROM CAST(creation_date AS date)) AS weeks
FROM stackoverflow.posts sp
JOIN
(SELECT COUNT(id),
       user_id
FROM stackoverflow.posts
GROUP BY 2
ORDER BY 1 DESC
LIMIT 1) AS u ON sp.user_id = u.user_id
GROUP BY 1,2)
SELECT r.weeks,
       MAX(r.creation_date)
FROM r
WHERE CAST(r.creation_date AS date) BETWEEN '2008-10-01' AND '2008-10-31'
GROUP BY 1
'''
result_20 = pd.read_sql(query_20, con = engine)
print(result_20)

   weeks                 max
0   40.0 2008-10-05 09:00:58
1   41.0 2008-10-12 21:22:23
2   42.0 2008-10-19 06:49:30
3   44.0 2008-10-31 22:16:01
4   43.0 2008-10-26 21:44:36


Комментарий


## Вывод


Мы создали коннекцию к база данных и выгрузили таблицы:

 - badges
 - post_types
 - posts
 - users
 - vote_types
 - votes
 
Изучили данные, проверили на дубликаты. Типы данных представлены в корректном виде. Пропуски оставили без изменений.
 
Написали SQL- запросы для ответов на вопросы задания. Результаты запросов прошли проверку тренажера.
